# CDFS-SWIRE Master List Creation

## Preparation of VIDEO/VISTA/VIRCAM data


The catalogue comes from `dmu0_VISTA_VIDEO-private`.

There is an old public version of the catalogue but we are using the newer private version in the hope that it will be public by the time we publish the masterlist.

Filters: Z,Y,J,H,K 

In the catalogue, we keep:

- The identifier (it's unique in the catalogue);
- The position (degrees);
- The stellarity;
- The magnitude for each band in apertude 3, which is 2 arcsec (rs548 presumes same for private catalogue).
- The “auto” magnitude is provided, we presume this is standard Sextractor units etc.

Yannick said the dates of observation for VIDEO are from 2009/11 to 2016/12. There is a paper from 2012 (Jarvis et al). So will use 2012.



In [1]:
from herschelhelp_internal import git_version
print("This notebook was run with herschelhelp_internal version: \n{}".format(git_version()))

This notebook was run with herschelhelp_internal version: 
c40aaa9 (Wed May 31 17:58:17 2017 +0100) [with local modifications]


In [2]:
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))

from astropy.table import Column, Table
from astropy import units as u
from astropy.coordinates import SkyCoord
import matplotlib as mpl
import numpy as np
import seaborn as sns
from pymoc import MOC
from matplotlib_venn import venn3

from herschelhelp_internal import flagging, utils, masterlist, starmask

/Users/rs548/anaconda/envs/herschelhelp_internal/lib/python3.6/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [3]:
import locale
locale.setlocale(locale.LC_ALL, 'en_GB')

'en_GB'

In [4]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [ ]:
OUT_DIR =  os.environ.get('TMP_DIR', "./data_tmp")
try:
    os.makedirs(OUT_DIR)
except FileExistsError:
    pass

RA_COL = "wfc_ra"
DEC_COL = "wfc_dec"

## I - Column selection

In [ ]:
imported_columns = OrderedDict({
        'ID':              'video_id', 
        'ALPHA_J2000':     'video_ra', 
        'DELTA_J2000':     'video_dec', 
        'J_CLASS_STAR':    'video_stellarity', 
        'Z_MAG_APER_3':    'm_ap_video_z', 
        'Z_MAGERR_APER_3': 'merr_ap_video_z', 
        'Z_MAG_AUTO':      'm_video_z', 
        'Z_MAGERR_AUTO':   'merr_video_z',
        'Z_FLUX_APER_3':   'f_ap_video_z', 
        'Z_FLUXERR_APER_3':'ferr_ap_video_z', 
        'Z_FLUX_AUTO':     'f_video_z', 
        'Z_FLUXERR_AUTO':  'ferr_video_z',
        'Y_MAG_APER_3':    'm_ap_video_y', 
        'Y_MAGERR_APER_3': 'merr_ap_video_y', 
        'Y_MAG_AUTO':      'm_video_y', 
        'Y_MAGERR_AUTO':   'merr_video_y',
        'Y_FLUX_APER_3':   'f_ap_video_y', 
        'Y_FLUXERR_APER_3':'ferr_ap_video_y', 
        'Y_FLUX_AUTO':     'f_video_y', 
        'Y_FLUXERR_AUTO':  'ferr_video_y',
        'J_MAG_APER_3':    'm_ap_video_j', 
        'J_MAGERR_APER_3': 'merr_ap_video_j', 
        'J_MAG_AUTO':      'm_video_j', 
        'J_MAGERR_AUTO':   'merr_video_j',
        'J_FLUX_APER_3':   'f_ap_video_j', 
        'J_FLUXERR_APER_3':'ferr_ap_video_j', 
        'J_FLUX_AUTO':     'f_video_j', 
        'J_FLUXERR_AUTO':  'ferr_video_j',
        'H_MAG_APER_3':    'm_ap_video_h', 
        'H_MAGERR_APER_3': 'merr_ap_video_h', 
        'H_MAG_AUTO':      'm_video_h', 
        'H_MAGERR_AUTO':   'merr_video_h',
        'H_FLUX_APER_3':   'f_ap_video_h', 
        'H_FLUXERR_APER_3':'ferr_ap_video_h', 
        'H_FLUX_AUTO':     'f_video_h', 
        'H_FLUXERR_AUTO':  'ferr_video_h',    
        'K_MAG_APER_3':    'm_ap_video_k', 
        'K_MAGERR_APER_3': 'merr_ap_video_k', 
        'K_MAG_AUTO':      'm_video_k', 
        'K_MAGERR_AUTO':   'merr_video_k',
        'K_FLUX_APER_3':   'f_ap_video_k', 
        'K_FLUXERR_APER_3':'ferr_ap_video_k', 
        'K_FLUX_AUTO':     'f_video_k', 
        'K_FLUXERR_AUTO':  'ferr_video_k'
})


catalogue = Table.read("../../dmu0/dmu0_VISTA-VIDEO-private/data/VIDEO-all_2016-04-14_fullcat_errfix_CDFS-SWIRE.fits"
                      )[list(imported_columns)]
for column in imported_columns:
    catalogue[column].name = imported_columns[column]
    
    
video_epoch = 2012
#MOC made using stilts: stilts pixfoot in=VIDEO-all_2016-04-14_fullcat_errfix_ELAIS-S1.fits
#out=VIDEO-all_2016-04-14_fullcat_errfix_ELAIS-S1_MOC.fits ra=ALPHA_J2000 dec=DELTA_J2000 order=13
video_moc = MOC(filename="../../dmu0/dmu0_VISTA-VIDEO-private/data/VIDEO-all_2016-04-14_fullcat_errfix_CDFS-SWIRE_MOC.fits")

# Adding band-flag columns
for col in video.colnames:
    if col.startswith('m_'):
        
        #errcol = "merr{}".format(col[1:])
        #flux, error = utils.mag_to_flux(np.array(video[col]), np.array(video[errcol]))

        # Fluxes are added in µJy
        #video.add_column(Column(flux * 1.e6, name="f{}".format(col[1:])))
        #video.add_column(Column(error * 1.e6, name="f{}".format(errcol[1:])))
        
        # Band-flag column
        video.add_column(Column(np.zeros(len(video), dtype=bool), name="flag{}".format(col[1:])))
        
# TODO: Set to True the flag columns for fluxes that should not be used for SED fitting.


In [6]:
#Both flux and mag available in pristine catalogues
#I think these are still 2 arcsec aperture phtometriers and need to be replaceed with 3 arcsec.
#There is a PSTAR for each band and they vary quite a bit. I took the J band because it is in the middle
video = Table.read("../../dmu0/dmu0_VISTA-VIDEO-private/data/VIDEO-all_2016-04-14_fullcat_errfix_CDFS-SWIRE.fits")[
    'ID', 'ALPHA_J2000', 'DELTA_J2000', 'J_CLASS_STAR',     
    'Z_MAG_APER_3', 'Z_MAGERR_APER_3', 'Z_MAG_AUTO', 'Z_MAGERR_AUTO',
    'Z_FLUX_APER_3', 'Z_FLUXERR_APER_3', 'Z_FLUX_AUTO', 'Z_FLUXERR_AUTO',
    'Y_MAG_APER_3', 'Y_MAGERR_APER_3', 'Y_MAG_AUTO', 'Y_MAGERR_AUTO',
    'Y_FLUX_APER_3', 'Y_FLUXERR_APER_3', 'Y_FLUX_AUTO', 'Y_FLUXERR_AUTO',
    'J_MAG_APER_3', 'J_MAGERR_APER_3', 'J_MAG_AUTO', 'J_MAGERR_AUTO',
    'J_FLUX_APER_3', 'J_FLUXERR_APER_3', 'J_FLUX_AUTO', 'J_FLUXERR_AUTO',
    'H_MAG_APER_3', 'H_MAGERR_APER_3', 'H_MAG_AUTO', 'H_MAGERR_AUTO',
    'H_FLUX_APER_3', 'H_FLUXERR_APER_3', 'H_FLUX_AUTO', 'H_FLUXERR_AUTO',
    'K_MAG_APER_3', 'K_MAGERR_APER_3', 'K_MAG_AUTO', 'K_MAGERR_AUTO',
    'K_FLUX_APER_3', 'K_FLUXERR_APER_3', 'K_FLUX_AUTO', 'K_FLUXERR_AUTO']
video = Table(video.as_array(), names=[
    'video_id', 'video_ra', 'video_dec', 'video_stellarity',
    'm_ap_video_z', 'merr_ap_video_z', 'm_video_z', 'merr_video_z',
    'f_ap_video_z', 'ferr_ap_video_z', 'f_video_z', 'ferr_video_z',
    'm_ap_video_y', 'merr_ap_video_y', 'm_video_y', 'merr_video_y',
    'f_ap_video_y', 'ferr_ap_video_y', 'f_video_y', 'ferr_video_y',
    'm_ap_video_j', 'merr_ap_video_j', 'm_video_j', 'merr_video_j',
    'f_ap_video_j', 'ferr_ap_video_j', 'f_video_j', 'ferr_video_j',
    'm_ap_video_h', 'merr_ap_video_h', 'm_video_h', 'merr_video_h',
    'f_ap_video_h', 'ferr_ap_video_h', 'f_video_h', 'ferr_video_h',
    'm_ap_video_k', 'merr_ap_video_k', 'm_video_k', 'merr_video_k',
    'f_ap_video_k', 'ferr_ap_video_k', 'f_video_k', 'ferr_video_k',
])

video_epoch = 2012
#MOC made using stilts: stilts pixfoot in=VIDEO-all_2016-04-14_fullcat_errfix_ELAIS-S1.fits
#out=VIDEO-all_2016-04-14_fullcat_errfix_ELAIS-S1_MOC.fits ra=ALPHA_J2000 dec=DELTA_J2000 order=13
video_moc = MOC(filename="../../dmu0/dmu0_VISTA-VIDEO-private/data/VIDEO-all_2016-04-14_fullcat_errfix_CDFS-SWIRE_MOC.fits")

# Adding band-flag columns
for col in video.colnames:
    if col.startswith('m_'):
        
        #errcol = "merr{}".format(col[1:])
        #flux, error = utils.mag_to_flux(np.array(video[col]), np.array(video[errcol]))

        # Fluxes are added in µJy
        #video.add_column(Column(flux * 1.e6, name="f{}".format(col[1:])))
        #video.add_column(Column(error * 1.e6, name="f{}".format(errcol[1:])))
        
        # Band-flag column
        video.add_column(Column(np.zeros(len(video), dtype=bool), name="flag{}".format(col[1:])))
        
# TODO: Set to True the flag columns for fluxes that should not be used for SED fitting.

In [7]:
video[:10].show_in_notebook()

idx,video_id,video_ra,video_dec,video_stellarity,m_app_video_z,merr_app_video_z,m_video_z,merr_video_z,f_app_video_z,ferr_app_video_z,f_video_z,ferr_video_z,m_app_video_y,merr_app_video_y,m_video_y,merr_video_y,f_app_video_y,ferr_app_video_y,f_video_y,ferr_video_y,m_app_video_j,merr_app_video_j,m_video_j,merr_video_j,f_app_video_j,ferr_app_video_j,f_video_j,ferr_video_j,m_app_video_h,merr_app_video_h,m_video_h,merr_video_h,f_app_video_h,ferr_app_video_h,f_video_h,ferr_video_h,m_app_video_k,merr_app_video_k,m_video_k,merr_video_k,f_app_video_k,ferr_app_video_k,f_video_k,ferr_video_k,flag_app_video_z,flag_video_z,flag_app_video_y,flag_video_y,flag_app_video_j,flag_video_j,flag_app_video_h,flag_video_h,flag_app_video_k,flag_video_k
0,1307519,52.5764425036,-28.0036165902,0.0103968,23.1244,0.0856302,22.9351,0.138891,562.637,44.3758,669.844,85.6917,22.8123,0.087532,22.6132,0.145146,750.06,60.4718,900.997,120.453,22.5619,0.0792798,22.4394,0.137883,944.59,68.9757,1057.4,134.289,22.2869,0.0997219,22.1966,0.17553,1216.91,111.774,1322.44,213.805,21.6483,0.0821624,21.5571,0.143083,2191.18,165.822,2383.2,314.079,False,False,False,False,False,False,False,False,False,False
1,1307537,52.5603041414,-28.0034513965,0.0729567,23.9185,0.177761,23.9029,0.336246,270.763,44.332,274.681,85.07,23.5323,0.169923,23.5279,0.33662,386.446,60.4827,388.002,120.299,23.5371,0.194704,23.4868,0.360703,384.726,68.995,402.968,133.878,23.2214,0.235867,23.1842,0.434417,514.565,111.789,532.494,213.065,22.9315,0.267905,22.9023,0.492546,672.046,165.832,690.365,313.196,False,False,False,False,False,False,False,False,False,False
2,1307545,52.56806518,-28.0034720553,0.0404727,21.4484,0.0182901,21.2843,0.0305287,2634.17,44.3761,3064.05,86.1577,21.2754,0.0212568,21.1132,0.0365021,3089.02,60.4795,3586.77,120.59,21.0487,0.0196787,20.9174,0.0340309,3806.45,68.9933,4295.74,134.649,20.828,0.0260207,20.752,0.0465374,4664.45,111.792,5002.51,214.427,20.6027,0.0313811,20.4955,0.053958,5740.28,165.917,6336.06,314.895,False,False,False,False,False,False,False,False,False,False
3,1307547,52.6574343795,-28.0038182732,0.00168766,26.6988,2.30199,26.9262,5.41295,20.9152,44.3462,16.9637,84.5755,24.5844,0.447751,24.6243,0.922916,146.635,60.4735,141.349,120.156,24.1176,0.332201,24.2266,0.710945,225.41,68.9706,203.871,133.5,22.8995,0.175361,22.9562,0.351068,692.145,111.795,656.931,212.423,22.6307,0.203072,22.6578,0.392258,886.622,165.836,864.739,312.426,False,False,False,False,False,False,False,False,False,False
4,1307553,52.7597187859,-28.0042276187,0.762026,23.2616,0.0954313,23.4266,0.214152,495.872,43.5863,425.969,84.0214,23.0878,0.112845,23.2196,0.252911,581.936,60.4851,515.424,120.067,23.1501,0.136275,23.1841,0.271659,549.468,68.9681,532.567,133.257,22.7953,0.158786,22.8781,0.32586,761.873,111.426,705.945,211.881,22.956,0.274104,22.9585,0.5166,657.057,165.885,655.539,311.92,False,False,False,False,False,False,False,False,False,False
5,1307559,52.6922441369,-28.0039817321,0.981548,21.4363,0.0180672,21.4903,0.0361918,2663.65,44.3259,2534.33,84.4819,21.0571,0.0173892,21.1005,0.0359413,3777.21,60.498,3629.2,120.142,20.8283,0.016076,20.8697,0.0322816,4663.03,69.0455,4488.85,133.469,20.8261,0.0259885,20.854,0.0506186,4672.75,111.852,4554.28,212.334,20.9607,0.0436281,20.9933,0.0846454,4127.81,165.873,4005.8,312.308,False,False,False,False,False,False,False,False,False,False
6,1307566,52.5415573606,-28.0032513622,0.00021876,23.2598,0.0957977,23.0407,0.153296,496.683,43.8253,607.726,85.8083,23.3198,0.139642,23.2777,0.26807,469.998,60.4509,488.565,120.631,22.8386,0.102236,23.0488,0.242567,732.049,68.9342,603.21,134.769,22.6178,0.134811,22.4296,0.218087,897.213,111.406,1067.0,214.33,22.4988,0.179821,22.5501,0.358374,1001.07,165.804,954.879,315.192,False,False,False,False,False,False,False,False,False,False
7,1307567,52.7803303559,-28.0042248642,0.0792431,25.1813,0.559335,26.1469,2.62526,84.6218,43.5958,34.7742,84.0851,24.8803,0.587909,24.9757,1.27495,111.659,60.4635,102.259,120.084,24.4588,0.454699,2

## II - Removal of duplicated sources

We remove duplicated objects from the input catalogues.

In [38]:
nb_video_orig = len(video)
video = masterlist.remove_duplicates(
    video, 'video_ra', 'video_dec', 
    sort_col=['merr_ap_video_z', 'merr_ap_video_y', 'merr_ap_video_j', 'merr_ap_video_h', 'merr_ap_video_k'],
    flag_name='video_flag_cleaned')
nb_video = len(video)
print("VIDEO initial catalogue as {} sources.".format(nb_video_orig))
print("VIDEO cleaned catalogue as {} sources ({} removed).".format(nb_video, nb_video_orig - nb_video))
print("VIDEO has {} sources flagged as having been cleaned".format(np.sum(video['video_flag_cleaned'])))

VIDEO initial catalogue as 190847 sources.
VIDEO cleaned catalogue as 190396 sources (451 removed).
VIDEO has 449 sources flagged as having been cleaned


## III - Astrometry correction

We match the astrometry to the Gaia one. We limit the Gaia catalogue to sources with a g band flux between the 30th and the 70th percentile. Some quick tests show that this give the lower dispersion in the results.

In [47]:
gaia = Table.read("../../dmu0/dmu0_GAIA/data/GAIA_CDFS-SWIRE.fits")
gaia_coords = SkyCoord(gaia['ra'], gaia['dec'])

In [48]:
masterlist.nb_astcor_diag_plot(
    video['video_ra'], video['video_dec'], gaia_coords.ra, gaia_coords.dec)

NameError: name 'wfc' is not defined

In [ ]:
video_delta_ra, video_delta_dec = utils.astrometric_correction(
    SkyCoord(video['video_ra'], video['video_dec']),
    gaia_coords
)
video['video_ra'] += video_delta_ra.to(u.deg)
video['wfc_dec'] += video_delta_dec.to(u.deg)

print("VIDEO delta RA / delta Dec: {} / {}".format(video_delta_ra, video_delta_dec))

## IV - Flagging Gaia objects

In [ ]:
catalogue.add_column(
    flagging.gaia_flag_column(
        SkyCoord(wfc['video_ra'], wfc['video_dec']),
        epoch,
        gaia
    )
)
catalogue['flag_gaia'].name = 'video_flag_gaia'
print("{} sources flagged.".format(np.sum(catalogue['video_flag_gaia'] > 0)))

## IV - Flagging objects near bright stars

In [ ]:
#This section may be superseeded by merely using multiband detection as evidence of good data
#starmask.flag_artefacts

## V - Saving to disk

In [ ]:
catalogue.write("{}/VIDEO.fits".format(OUT_DIR), overwrite=True)